In [1]:
library(MASS)
library(dplyr)
library(tidyr)
library(ggplot2)
library(ascii)



Attaching package: ‘dplyr’

The following object is masked from ‘package:MASS’:

    select

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘ascii’

The following object is masked from ‘package:tidyr’:

    expand



In [2]:
# Had to install this from the console
# install.packages("lubridate",dependencies="TRUE")

In [3]:
library(splines)


In [4]:
library(mgcv)

Loading required package: nlme

Attaching package: ‘nlme’

The following object is masked from ‘package:dplyr’:

    collapse

This is mgcv 1.8-23. For overview type 'help("mgcv-package")'.


In [5]:
install.packages("lubridate")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("lubridate"):
“installation of package ‘lubridate’ had non-zero exit status”

In [6]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [7]:
PSDS_PATH <- file.path('~')

In [8]:
lung <- read.csv(file.path(PSDS_PATH, 'LungDisease.csv'))

zhvi <- read.csv(file.path(PSDS_PATH, 'County_Zhvi_AllHomes.csv'))
zhvi <- unlist(zhvi[13,-(1:5)])

In [9]:
dates <- parse_date_time(paste(substr(names(zhvi), start=2, stop=8), "01", sep="."), "Ymd")
zhvi <- data.frame(ym=dates, zhvi_px=zhvi, row.names = NULL) %>%
  mutate(zhvi_idx=zhvi/last(zhvi))

house <- read.csv(file.path(PSDS_PATH, 'house_sales.csv'), sep='\t')

In [10]:
plot(lung$Exposure, lung$PEFR, xlab="Exposure", ylab="PEFR")
dev.off()

null device 
          1

In [11]:
model <- lm(PEFR ~ Exposure, data=lung)
model


Call:
lm(formula = PEFR ~ Exposure, data = lung)

Coefficients:
(Intercept)     Exposure  
    424.583       -4.185  


In [12]:
plot(lung$Exposure, lung$PEFR, xlab="Exposure", ylab="PEFR", ylim=c(300,450), type="n", xaxs="i")
abline(a=model$coefficients[1], b=model$coefficients[2], col="blue", lwd=2)
text(x=.3, y=model$coefficients[1], labels=expression("b"[0]),  adj=0, cex=1.5)
x <- c(7.5, 17.5)
y <- predict(model, newdata=data.frame(Exposure=x))
segments(x[1], y[2], x[2], y[2] , col="red", lwd=2, lty=2)
segments(x[1], y[1], x[1], y[2] , col="red", lwd=2, lty=2)
text(x[1], mean(y), labels=expression(Delta~Y), pos=2, cex=1.5)
text(mean(x), y[2], labels=expression(Delta~X), pos=1, cex=1.5)
text(mean(x), 400, labels=expression(b[1] == frac(Delta ~ Y, Delta ~ X)), cex=1.5)
dev.off()

null device 
          1

In [13]:
fitted = predict(model)

In [14]:
fitted

1        2        3        4        5        6        7        8 
424.5828 424.5828 424.5828 424.5828 420.3982 416.2137 416.2137 416.2137 
       9       10       11       12       13       14       15       16 
412.0291 412.0291 412.0291 412.0291 412.0291 412.0291 407.8445 407.8445 
      17       18       19       20       21       22       23       24 
407.8445 407.8445 407.8445 407.8445 403.6599 399.4753 399.4753 399.4753 
      25       26       27       28       29       30       31       32 
399.4753 399.4753 395.2908 395.2908 395.2908 395.2908 395.2908 395.2908 
      33       34       35       36       37       38       39       40 
395.2908 391.1062 386.9216 382.7370 382.7370 382.7370 374.3679 374.3679 
      41       42       43       44       45       46       47       48 
370.1833 370.1833 370.1833 370.1833 370.1833 370.1833 370.1833 370.1833 
      49       50       51       52       53       54       55       56 
365.9987 365.9987 365.9987 365.9987 365.9987 365.9987 361.8142 361.8142 
      57       58       59       60       61       62       63       64 
361.8142 357.6296 357.6296 353.4450 353.4450 353.4450 353.4450 353.4450 
      65       66       67       68       69       70       71       72 
353.4450 349.2604 349.2604 349.2604 349.2604 345.0759 345.0759 345.0759 
      73       74       75       76       77       78       79       80 
345.0759 345.0759 345.0759 345.0759 345.0759 345.0759 345.0759 345.0759 
      81       82       83       84       85       86       87       88 
345.0759 345.0759 340.8913 340.8913 340.8913 340.8913 340.8913 340.8913 
      89       90       91       92       93       94       95       96 
340.8913 340.8913 340.8913 340.8913 340.8913 340.8913 340.8913 340.8913 
      97       98       99      100      101      102      103      104 
340.8913 340.8913 340.8913 340.8913 340.8913 340.8913 340.8913 340.8913 
     105      106      107      108      109      110      111      112 
336.7067 336.7067 336.7067 336.7067 336.7067 336.7067 336.7067 336.7067 
     113      114      115      116      117      118      119      120 
336.7067 336.7067 332.5221 332.5221 332.5221 332.5221 332.5221 332.5221 
     121      122 
328.3375 328.3375

In [15]:
lung

PEFR,Exposure
390,0
410,0
430,0
460,0
420,1
280,2
420,2
520,2
610,3
590,3


In [16]:
# set plot parameters (margin)
par(mar=c(4,4,0,0)+.1)

#group by Exposure showing max and min PEFR foreach exposure level. Then show a column with the 'fitted' value (from the model) for PEFR
lung1 <- lung %>%
  mutate(Fitted=fitted,
         positive = PEFR>Fitted) %>%
  group_by(Exposure, positive) %>%
  summarize(PEFR_max = max(PEFR), 
            PEFR_min = min(PEFR),
            Fitted = first(Fitted)) %>%
  ungroup() %>%
  mutate(PEFR = ifelse(positive, PEFR_max, PEFR_min)) %>%
  arrange(Exposure)

plot(lung$Exposure, lung$PEFR, xlab="Exposure", ylab="PEFR")
abline(a=model$coefficients[1], b=model$coefficients[2], col="blue", lwd=2)
segments(lung1$Exposure, lung1$PEFR, lung1$Exposure, lung1$Fitted, col="red", lty=3)

In [17]:
lung1

Exposure,positive,PEFR_max,PEFR_min,Fitted,PEFR
0,FALSE,410,390,424.5828,390
0,TRUE,460,430,424.5828,460
1,FALSE,420,420,420.3982,420
2,FALSE,280,280,416.2137,280
2,TRUE,520,420,416.2137,520
3,FALSE,410,320,412.0291,320
3,TRUE,610,430,412.0291,610
4,FALSE,400,110,407.8445,110
4,TRUE,520,420,407.8445,520
5,TRUE,460,460,403.6599,460


In [18]:
rsid = residuals(model)

In [19]:
# residuals, how far off were data points from their predicted values.
rsid

1            2            3            4            5            6 
 -34.5828066  -14.5828066    5.4171934   35.4171934   -0.3982301 -136.2136536 
           7            8            9           10           11           12 
   3.7863464  103.7863464  197.9709229  177.9709229   17.9709229   -2.0290771 
          13           14           15           16           17           18 
 -52.0290771  -92.0290771 -297.8445006  -17.8445006   -7.8445006   12.1554994 
          19           20           21           22           23           24 
  42.1554994  112.1554994   56.3400759 -199.4753477   -9.4753477  -59.4753477 
          25           26           27           28           29           30 
  50.5246523  150.5246523  -95.2907712  -75.2907712  -55.2907712  -35.2907712 
          31           32           33           34           35           36 
  34.7092288   84.7092288  164.7092288  108.8938053  -76.9216182   47.2629583 
          37           38           39           40           41           42 
 117.2629583  127.2629583 -124.3678887   35.6321113 -150.1833123  -70.1833123 
          43           44           45           46           47           48 
 -30.1833123  -10.1833123   29.8166877   79.8166877  119.8166877  129.8166877 
          49           50           51           52           53           54 
-165.9987358 -145.9987358  -45.9987358   -5.9987358   44.0012642   54.0012642 
          55           56           57           58           59           60 
  18.1858407   48.1858407   88.1858407 -137.6295828  -37.6295828 -223.4450063 
          61           62           63           64           65           66 
-133.4450063  -33.4450063    6.5549937    6.5549937  136.5549937 -129.2604298 
          67           68           69           70           71           72 
-119.2604298  -39.2604298  110.7395702 -185.0758534 -145.0758534 -125.0758534 
          73           74           75           76           77           78 
 -65.0758534  -55.0758534  -35.0758534  -25.0758534  -15.0758534   14.9241466 
          79           80           81           82           83           84 
  44.9241466   54.9241466   84.9241466  164.9241466 -190.8912769 -150.8912769 
          85           86           87           88           89           90 
-140.8912769 -130.8912769 -100.8912769 -120.8912769  -90.8912769  -80.8912769 
          91           92           93           94           95           96 
 -60.8912769  -50.8912769  -40.8912769  -30.8912769  -20.8912769  -10.8912769 
          97           98           99          100          101          102 
  19.1087231   49.1087231   59.1087231   89.1087231  119.1087231  129.1087231 
         103          104          105          106          107          108 
 149.1087231  209.1087231  -56.7067004  -46.7067004  -16.7067004   -6.7067004 
         109          110          111          112          113          114 
  13.2932996   33.2932996   53.2932996   63.2932996  173.2932996  193.2932996 
         115          116          117          118          119          120 
 -52.5221239   27.4778761   67.4778761  117.4778761  157.4778761  167.4778761 
         121          122 
  41.6624526   61.6624526